# Robust Knapsack

This tutorial shows how to construct and solve the Robust knapsack optimization problem (Bertsimas and Simm 2004, (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065). The problem can be formulated as follows:
$$
\begin{align}
\max_{\mathbf{x}} \;&\mathbf{c}^T\mathbf{x} \nonumber \\

\text{s.t. } &\left( \mathbf{w} + \mathbf{\Delta}\mathbf{z} \right)^T\mathbf{x} \leq b \nonumber \\

&\mathbf{x}\in \left\{ 0,1 \right\}^n \nonumber \\

&\mathbf{z}\in \left\{\mathbf{z}:  \left \lVert \mathbf{z} \right \rVert_{\infty} \leq 1, \left \lVert \mathbf{z} \right \rVert_{1} \leq r \right\} \nonumber \\
\end{align}
$$

where the items values are denoted by $\mathbf{c}$, and the uncertainty is represented by $\mathbf{z}$. The matrix $\Delta$ is a diagonal matrix. The vector $\mathbf{w}$ stores the nominal weight coefficients.

In [1]:
import cvxpy as cp
import numpy as np

from lropt import Budget
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We define the constants as shown below:

In [2]:
N = 50                                             # Number of items
b = 2000                                           # Constraint constant

c = 2*np.random.randint(low=5, high=10, size=N)    # Profit coefficients
w = 2*np.random.randint(low=10, high=41, size=N)   # Nominal weights
delta = np.diag(0.2*w)                             # Maximum deviations
r = 2                                              # Uncertainty budget

The problem is formulated and solved in the block below, using the Budget uncertainty set which is defined as follows:

Budget uncertainty: $\{u \mid \| A_1u + b_1 \|_\infty \leq \rho_1, \| A_2u + b_2 \|_1 \leq \rho_2\}$ 

- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.
- $A_1$, $A_2$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b_1$, $b_2$ : np.array, optional
  * Relocation vector for u. Default None.

In [3]:
uncertainty_set = Budget(rho1=1, rho2=r)                            # Budget uncertainty set
x = cp.Variable(N, boolean=True)                                    # Boolean optimization variables, control which item is taken
z = UncertainParameter(N, uncertainty_set=uncertainty_set)          # Uncertain parameters, control the weight coefficients

objective = cp.Maximize(c@x)                                        # Problem objective
constraints = [
                (w + delta@z)@x <= b,
              ]                                                     # Problem constraints
prob = RobustProblem(objective=objective, constraints=constraints)  # Define the problem using LROPT 
prob.solve()                                                        # Solve the problem using LROPT

626.0